In [1]:
# connect google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import library

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

# Part1: goodFeaturesToTrack
- Fill the missing part (denoted as ```fill here```) of the code
- We provide procedure comments for complete the function

In [3]:
def goodFeaturesToTrack(image, maxCorners=100, qualityLevel=0.03, blocksize=7):

    # Image bluring wih averaging filter
    # Only cv2.filter2D is allowed for convolution operation!
    '''fill here'''
    avg_filter = (1/(blocksize**2))*np.ones((blocksize, blocksize))
    #avg_filter = (1/49)*np.ones((7, 7))
    image = cv2.filter2D(image, -1, avg_filter)

    # Compute gradients
    '''fill here'''
    sobel_x = np.array([[-1/8, 0, 1/8], [-2/8, 0, 2/8], [-1/8, 0, 1/8]])
    sobel_y = np.array([[-1/8, -2/8, -1/8], [0, 0, 0], [1/8, 2/8, 1/8]])
    #prewitt_x = np.array([[-1/6, 0, 1/6], [-1/6, 0, 1/6], [-1/6, 0, 1/6]])
    #prewitt_y = np.array([[-1/6, -1/6, -1/6], [0, 0, 0], [1/6, 1/6, 1/6]])
    Ix = cv2.filter2D(image, -1, sobel_x)
    Iy = cv2.filter2D(image, -1, sobel_y)

    # Compute products of gradients at each pixel
    '''fill here'''
    Ixx = Ix * Ix
    Iyy = Iy * Iy
    Ixy = Ix * Iy

    # Compute the sums of products of gradients in local windows
    '''fill here'''
    offset = int(np.floor(blocksize/2))
    Sxx = np.zeros(image.shape)
    Syy = np.zeros(image.shape)
    Sxy = np.zeros(image.shape)
    for row in range(offset, image.shape[0]-offset):
        for col in range(offset, image.shape[1]-offset):
            Sxx[row, col] = np.sum(Ixx[row-offset:row+offset+1, col-offset:col+offset+1])
            Syy[row, col] = np.sum(Iyy[row-offset:row+offset+1, col-offset:col+offset+1])
            Sxy[row, col] = np.sum(Ixy[row-offset:row+offset+1, col-offset:col+offset+1])


    # Compute the determinant and trace of the matrix M for each pixel
    '''fill here'''
    detM = (Sxx*Syy) - (Sxy**2)
    traceM = Sxx + Syy

    # Compute the Harris response with detM and traceM
    '''fill here'''
    #harris_response = detM - 0.04 * (traceM ** 2)
    harris_response = detM / (traceM + 1e-5)

    # Threshold the Harris response to find candidate corners
    '''fill here'''
    #corners = np.argwhere(harris_response > qualityLevel * np.max(harris_response))
    corners = np.argwhere(harris_response > qualityLevel)

    # Sort the corners by Harris response in descending order
    sorted_corners = corners[np.argsort(harris_response[corners[:, 0], corners[:, 1]])[::-1]]

    # Keep the top 'maxCorners' corners
    selected_corners = sorted_corners[:maxCorners]

    final_corners = np.array(selected_corners)
    final_corners = final_corners.reshape(-1, 1, 2)

    return final_corners

# Part2: Optical flow with Lukas-Kanade
- Fill the missing part (denoted as ```fill here```) of the code
- We provide procedure comments for complete the function

In [4]:

def optical_flow(old_frame, new_frame, window_size, min_quality):

    feature_list = goodFeaturesToTrack(old_frame, max_corners, min_quality, blocksize)

    w = int(window_size/2)

    # Normalize
    old_frame = old_frame / 255
    new_frame = new_frame / 255

    # Convolve to get gradients w.r.to X, Y and T dimensions
    '''fill here'''
    kernel_x = np.array([[-1/8, 0, 1/8], [-2/8, 0, 2/8], [-1/8, 0, 1/8]])
    kernel_y = np.array([[-1/8, -2/8, -1/8], [0, 0, 0], [1/8, 2/8, 1/8]])
    kernel_t = np.array([[1/9, 1/9, 1/9], [1/9, 1/9, 1/9], [1/9, 1/9, 1/9]])

    # cv2.filter2D is allowed for convolution!
    '''fill here'''
    fx =  cv2.filter2D(old_frame, -1, kernel_x)
    fy =  cv2.filter2D(old_frame, -1, kernel_y)
    ft =  cv2.filter2D(new_frame, -1, kernel_t) - cv2.filter2D(old_frame, -1, -kernel_t)

    u = np.zeros(old_frame.shape)
    v = np.zeros(old_frame.shape)

    for feature in feature_list:  # for every corner
        i, j = feature.ravel()  # get cordinates of the corners (i,j).
        i, j = int(i), int(j)  # i,j are floats initially so convert to integer type

        '''fill here'''
        I_x = fx[i-w:i+w+1, j-w:j+w+1].flatten()
        I_y = fy[i-w:i+w+1, j-w:j+w+1].flatten()
        I_t = ft[i-w:i+w+1, j-w:j+w+1].flatten()

        b = np.reshape(I_t, (I_t.shape[0], 1))
        A = np.vstack((I_x, I_y)).T

        '''fill here'''
        U = np.matmul(np.linalg.pinv(np.matmul(A.T, A)), np.matmul(A.T, b))  # Solving for (u,v) i.e., U

        u[i, j] = U[0][0]
        v[i, j] = U[1][0]

    return (u, v)


# Main function
- If part1 and part2 were filled properly, the 'output.avi' will be generated!
- For google colab, as cv2.imshow() is not provided, so please use cv2_imshow (google.colab.patches) instead  

In [5]:
cap = cv2.VideoCapture('/content/drive/MyDrive/Colab Notebooks/Computer Vision/slow/slow.mp4')

# Take first frame and find corners in it
ret, old_frame = cap.read()

# Width and height of the file to save
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

# 'output.mp4' will be generated!
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('output.mp4',  fourcc, 30.0, (int(width), int(height)))

old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

### old_frame => old_gray
# Shi Tomasi parameter
max_corners = 100
min_quality = 0.3
blocksize = 7
p0 = goodFeaturesToTrack(old_gray, max_corners, min_quality, blocksize)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

while(1):
    ret, current_frame = cap.read()
    if not ret:
        break
    frame_gray = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)

    # calculate optical flow
    U, V = optical_flow(old_gray, frame_gray, 15, 0.03)

    for i in range(current_frame.shape[0]):
        for j in range(current_frame.shape[1]):
            u, v = U[i][j], V[i][j]
            if u and v:
                mask = cv2.line(mask, (j, i), (int(round(j + u)), int(round(i + v))), (0, 255, 0), 2)
                frame = cv2.arrowedLine(current_frame, (j, i), (int(round(j + u)), int(round(i + v))), (0, 255, 0), thickness=2)
                current_frame = cv2.add(current_frame, mask)

    # Display the frame with optical flow vectors
    cv2_imshow(current_frame)
    out.write(current_frame)
    # Break the loop if 'Esc' key is pressed
    if cv2.waitKey(30) == 27:
        break

    # Set the current frame as the previous frame for the next iteration
    old_gray = frame_gray

# Release the video capture object
cap.release()
out.release()

# Close the plot window when done
plt.close()
cv2.destroyAllWindows()


Output hidden; open in https://colab.research.google.com to view.